In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [ ]:
import json
with open('../KG-FPQ-data/art_YN.json', 'r') as file:
    json_data = json.load(file)

T_q = []
F_q = []
for data in json_data:
    T_q.append(data['TPQ'])
    for key, value in data.items():
        if key.startswith("FPQ_"):
            F_q.append(value)

all_q = T_q + F_q

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Model identifier from Hugging Face
model_id = "meta-llama/Meta-Llama-3-8B"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map= "auto", # Auto-allocates GPU(s) if available
    torch_dtype=torch.float16 # Optional, but saves GPU memory
)

In [ ]:
responses = []
for q in all_q:
    inputs = tokenizer(q, return_tensors="pt").to(model.device)

    # Generate outputs
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
    )
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    responses.append(response)

In [ ]:
responses

In [ ]:
import json

with open('llama3_8b_orig.json', 'w') as file:
    json.dump(responses, file)

In [ ]:
import json
with open('./g_retri_gg.json', 'r') as file:
    detect = json.load(file)

responses = []
for i in range(len(all_q)):
    if 'Yes' in detect[i]:
        qq = all_q[i] + " This question contains false premise." 
    else:
        qq = all_q[i]
        
    inputs = tokenizer(qq, return_tensors="pt").to(model.device)
    # Generate outputs
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
    )
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    responses.append(response)